In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
%matplotlib inline
from fastai.imports import *
from fastai.structured import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from statistics import stdev
import math

In [ ]:
def rmse(actual, predicted):
    return np.sqrt(np.mean((actual - predicted)**2))

In [ ]:
class RandomForest:
    def __init__(self, sample_size, n_trees = 10 , min_samples_leaf = 5, max_depth = 4, max_features = None):
        self.n_trees, self.sample_size , self.min_samples_leaf, self.max_depth, self.max_features = n_trees, sample_size, min_samples_leaf, max_depth, max_features
        self.trees = [self.create_tree() for _ in range(self.n_trees)]
        
    def create_tree(self):
        
        return DecisionTree(min_samples_leaf = self.min_samples_leaf, max_depth = self.max_depth, max_features = self.max_features)
    
    def fit(self, X, y):    
        for tree in self.trees:
            random_idxs = np.random.permutation(X.shape[0])[:self.sample_size]
            tree.var_split(X.iloc[random_idxs, :], y[random_idxs]) 
    
    def predict(self, x):
        return np.mean([t.predict(x) for t in self.trees], axis = 0)

    #exploring feature importances
    def plot_pdp(self, X, y, feature_names, n_clusters = 0): pass
    
    def find_feature_importances(self, X, y): pass
    
    

In [ ]:
%load_ext cython

In [ ]:
%%cython
import math
import numpy as np
def stddev(cnt, values, sum1):
    return math.sqrt(math.fabs((values/cnt) - (sum1/cnt)**2))

class DecisionTree:
    
    def __init__(self, int min_samples_leaf = 3, int max_depth = 4, int level = 0, max_features = None, float parent_value = float('inf')):
        self.min_samples_leaf, self.max_depth = min_samples_leaf, max_depth
        self.score = float('inf')
        self.value = 0.0
        self.leftTree , self.rightTree, self.split_val, self.split_feature  = None, None, None, None
        self.level = level
        self.max_features = max_features
        self.parent_value  = parent_value
       
       
        
    @property 
    def is_leaf(self):
        return self.score == float('inf')
    
   
    def find_better_split(self, x, y, int ind):
    
        x1 = x.values[:, ind]
        inds = np.argsort(x1, axis = 0)
        sorted_y, sorted_x = y[inds], x1[inds]
        rhs_count, rhs_sum, rhs_sum2  = x.shape[0], sorted_y.sum(), (sorted_y**2).sum() 
        lhs_count, lhs_sum, lhs_sum2 = 0, 0., 0.
      #  print('for '+ str(x.columns[ind]))
        for i in range(0, x.shape[0]-self.min_samples_leaf+1):
            lhs_count, lhs_sum, lhs_sum2 = lhs_count+1, lhs_sum + sorted_y[i], lhs_sum2 + sorted_y[i]**2
            rhs_count, rhs_sum, rhs_sum2 = rhs_count-1, rhs_sum - sorted_y[i], rhs_sum2 - sorted_y[i]**2
            if i < self.min_samples_leaf - 1 or sorted_x[i] == sorted_x[i+1]:
                continue
            updated_score = ((lhs_count * stddev(lhs_count, lhs_sum2, lhs_sum)) + (rhs_count * stddev(rhs_count, rhs_sum2, rhs_sum)))/(x.shape[0])
            #print('score: '+ str(updated_score))
            if updated_score < self.score :
                self.score = updated_score
                self.split_feature = x.columns[ind]
                self.split_val = i
                self.value = (np.mean(y[:i])*i + np.mean(y[i:])*(x.shape[0] - i))/(x.shape[0])
        self.score = self.score
       
        
   
    def var_split(self, x, y):
        
        if x.shape[0] > self.min_samples_leaf and self.level < self.max_depth - 1:
            #find max_features to split on
             
            if self.max_features is not None:
                if self.max_features in ['auto', 'sqrt']:
                    self.max_features = int(math.sqrt(x.shape[1]))
                else:
                    if self.max_features == 'log2':
                        self.max_features = int(np.log(float(x.shape[1]))/np.log(2))
                    else:
                        if isinstance(self.max_features, float):
                            self.max_features = int(self.max_features * x.shape[1])
                        else:
                            self.max_features = x.shape[1]
            else:
                self.max_features = x.shape[1]
            self.max_features = int(self.max_features)
            feature_inds = np.random.permutation(x.shape[1])[:self.max_features] 
            #print('will split on features: '+str(feature_inds))
            feature_inds = [index for index in feature_inds if x.columns[index] != None]
            for ind in feature_inds:
                self.find_better_split(x, y, ind) 
           # print('split on '+ str(self.split_feature)+' with score '+str(self.score) + "at level : "+str(self.level))
            if self.parent_value == float('inf'):
                self.parent_value  = self.value
            x_lhs, x_rhs = x.iloc[:self.split_val,:], x.iloc[self.split_val:,:]
            self.leftTree = DecisionTree(min_samples_leaf = self.min_samples_leaf, max_depth = self.max_depth, level = self.level + 1, parent_value = self.parent_value)
            self.leftTree.var_split(x_lhs, y[:self.split_val])
            self.rightTree = DecisionTree(min_samples_leaf = self.min_samples_leaf, max_depth = self.max_depth, level = self.level + 1, parent_value = self.parent_value)
            self.rightTree.var_split(x_rhs, y[self.split_val:])
        
        else :
            self.score = float('inf')
            #print('at leaf node : '+str(y))
            y = [val for val in y if val != None]
            self.value = np.mean(y)

        
    def predict_row(self, row):      
        if self.is_leaf: 
            #print('result: ' + str(self.value))
            return self.value
        if row[self.split_feature] < self.split_val:
            return self.leftTree.predict_row(row)
        else:
            return self.rightTree.predict_row(row)
    
    def predict_row_for_ti(self, row, feat_contribs):
        
        if self.is_leaf: 
           return self.value, self.parent_value , feat_contribs
        if row[self.split_feature] < self.split_val:
            if self.split_feature in feat_contribs.keys():
                feat_contribs[self.split_feature] += self.leftTree.value - self.value
            else:
                feat_contribs[self.split_feature] = self.leftTree.value - self.value
            return self.leftTree.predict_row_for_ti(row, feat_contribs)
        else:
            if self.split_feature in feat_contribs.keys():
                feat_contribs[self.split_feature] += self.rightTree.value - self.value
            else:
                feat_contribs[self.split_feature] = self.rightTree.value - self.value
            return self.rightTree.predict_row_for_ti(row, feat_contribs)
           
    def predict(self, X):
        y_pred = []
        for row in range(X.shape[0]):
            y_pred.append(self.predict_row(X.iloc[row, :]))
        return y_pred
    
    def get_prediction_and_bias(self):
        
        return self.parent_value, self.value 
        
    def get_child_trees(self):
        return self.leftTree, self.rightTree
  
    def __repr__(self):
        return "score: " +str(self.score) + " avg: "+str(self.value) +  " split val: " + str(self.split_val) + " split feature : "+ str(self.split_feature)

In [ ]:
class TreeInterpreter:
    
    def predict(self, rf_model_tree, row):
            prediction , bias, contribs = rf_model_tree.predict_row_for_ti(row, {})
            print('prediction: '+str(prediction)+"bias: "+str(bias)+" contributions: "+str(contribs))
           
            return prediction, bias, contribs

In [ ]:
#class DecisionTree():
 #   def __init__()
train_data = pd.read_csv('../input/train.csv')

train_data, valid_data  = train_test_split(train_data, train_size = 0.7, random_state = 1)

train_cats(train_data)
apply_cats(valid_data, train_data)

X_train, y_train, _ = proc_df(train_data, 'SalePrice', max_n_cat = 7)
X_valid, y_valid, _ = proc_df(valid_data, 'SalePrice', max_n_cat = 7)

y_train = np.log(y_train)
y_valid = np.log(y_valid)

rf = RandomForestRegressor(n_jobs = -1)
rf.fit(X_train, y_train)
feat_imp = rf.feature_importances_
imp = sorted(zip(feat_imp, X_train.columns), reverse = True)
imp[:10]


In [ ]:
#X_train = X_train[['OverallQual', 'GrLivArea']]
rf_simple = RandomForestRegressor(n_estimators = 5, n_jobs = -1, random_state = 1, max_depth = 4)
rf_simple.fit(X_train, y_train)
print(rmse(y_train, rf_simple.predict(X_train)))
print(rmse(y_valid, rf_simple.predict(X_valid)))
draw_tree(rf_simple[0], df = X_train, ratio = 0.8)



In [ ]:
%prun
rf_mine = RandomForest(500, n_trees = 5, min_samples_leaf = 3, max_depth = 6, max_features = 0.5)
rf_mine.fit(X_train, y_train)
print(rf_mine.trees[4])
print(rf_mine.trees[0].leftTree.leftTree.leftTree.leftTree)
print(rmse(y_train, rf_mine.predict(X_train)))
print(rmse(y_valid, rf_mine.predict(X_valid)))
plt.scatter(y_valid, rf_mine.predict(X_valid))

Finding the confidence of a prediction by finding standard deviation of its group.

In [ ]:

y_pred = np.stack([t.predict(X_valid) for t in rf_mine.trees])
mean_pred, std_dev = np.mean(y_pred[:,0]), np.std(y_pred[:,0])
X_new = X_valid.copy()
X_new['mean_pred'] = mean_pred
X_new['std_dev_pred'] = std_dev
X_new.OverallQual.value_counts().plot.barh()

In [ ]:
X_new['SalePrice'] = y_valid
flds = ['OverallQual','SalePrice', 'mean_pred', 'std_dev_pred']
overallQual_props = X_new[flds].groupby(['OverallQual'], as_index = False).mean()
overallQual_props
overallQual_props.plot('OverallQual', 'std_dev_pred', 'barh')

Find feature importances

In [ ]:
%%cython
import numpy as np

def find_feature_importances(self, X, y):
    feat_imp = {}
    y_pred_old = self.predict(X)
    for ind in range(X.shape[1]):
        X_new = X.copy()
        np.random.shuffle(X_new.values[:, ind])
        y_pred = self.predict(X_new)
        feat_imp[X.columns[ind]] = np.fabs(np.sum(y_pred_old) - np.sum(y_pred))
        del X_new
    
    features, importances =zip(*sorted(feat_imp.items(), reverse=True))
    return features, importances
    
RandomForest.find_feature_importances = find_feature_importances
idxs = np.random.permutation(X_train.shape[0])[:50]
features, imp = find_feature_importances(rf_mine, X_train.iloc[idxs, :], y_train[idxs])

plt.barh(features[:10], imp[:10])



All predictions right now are the same value due to the data distribution, hence feature importances not shown.

Finding partial feature dependencies and also feature interactions, by keeping other levers constant

In [ ]:
%%cython
import numpy as np
import matplotlib.pyplot as plt
def plot_pdp(self, X, y, feature_name, n_clusters = 2):
    feature_id = -1
    y_pred_vals = []
    y_pred_mean = []
    for index in range(X.shape[1]):
        if X.columns[index] == feature_name:
            feature_id = index
    X_new = X.copy()
    for feature_val in np.unique(X.values[:, feature_id]):
        
        X_new.values[:, feature_id] = feature_val
        y_pred_val = self.predict(X_new)
        
        y_pred_vals.append(y_pred_val)
           
   # print("y_pred_vals : "+str(y_pred_vals))   
    del X_new    
   # plt.scatter(y, y_pred_mean)
    y_pred_randoms = np.random.normal(y_pred_vals)[:n_clusters]
  #  for y_pred in y_pred_randoms:
   #     plt.plot(X.values[:, feature_id], y_pred)
        
        
    y_pred_mean = np.mean(y_pred_vals, axis = 0)
    print("y_pred_mean: "+str(y_pred_mean))
    plt.plot(X.values[:, feature_id], y_pred_mean)
    plt.show()
        
RandomForest.plot_pdp = plot_pdp
idxs = np.random.permutation(X_train.shape[0])[:10]
plot_pdp(rf_mine, X_train.iloc[idxs, :], y_train[idxs], 'OverallQual')
        

In [ ]:
%%cython
import numpy as np
def plot_pdp_interaction(self, X, y, feature_names, n_clusters = 2):
    features_list = list(enumerate(X.columns))
    feature1 = -1; feature2 = -1
    for ind, feature_name in features_list:
        if feature_name == feature_names[0]:
            feature1 = ind
        if feature_name == feature_names[1]:
            feature2 = ind
        if feature1 > -1 and feature2 > -1:
            break
    y_pred_vals = []
    X_new = X.copy()
    feature1_list = []; feature2_list = []
    for feature1_val in np.unique(X_new.values[:, feature1]):
        feature1_list.append(feature1_val)
        for feature2_val in np.unique(X_new.values[:, feature2]):
            X_new = X.copy()
            X_new.values[:, feature1], X_new.values[:, feature2] = feature1_val, feature2_val
            y_pred = self.predict(X_new)
            y_pred_vals.append(y_pred)
            feature2_list.append(feature2_val)
    del X_new
    y_pred_mean = np.mean(y_pred_vals, axis = 0)
    #plt.plot(feature1_list, feature2_list, fill_between = y_pred_mean, legend = 'True')
    return y_pred_mean
    
idxs = np.random.permutation(X_train.shape[0])[:10]
X_sample, y_sample = X_train.iloc[idxs, :], y_train[idxs]
RandomForest.plot_pdp_interaction = plot_pdp_interaction
p = plot_pdp_interaction(rf_mine, X_sample, y_sample, ['OverallQual', 'LotArea'])
pdp.pdp_interact_plot(p,  ['OverallQual', 'LotArea'])    
#Need to get deeper into PDP library and plots    

        
        

Tree Interpreter and way 2 of Feature Importances. Waterfall Chart

In [ ]:
ti = TreeInterpreter()
pr, bias, contribs  = ti.predict(rf_mine.trees[0], X_valid.iloc[0, :])

TODO: Waterfall Chart and Feature Interactions, by avergaing trends of feature contributions. Also, feature importance by averaging contributions across all trees.

In [ ]:
#!git clone https://www.github.com/chrispaulca/waterfall.git
from waterfall import waterfall_chart
waterfall_chart.plot(list(contribs.keys()), list(contribs.values()))

**We shall now find feature importances by averaging the contributions of features across all the trees in the RF Model**
This ought to lead us towards feature interactions.

In [ ]:
avg_feat_imp = {}
for tree in rf_mine.trees:
    for row in range(X_valid.shape[0]):
        _,_, contribs  = ti.predict(tree, X_valid.iloc[row, :])
        for key, value in contribs.items():
            if key not in avg_feat_imp.keys():
                avg_feat_imp[key] = [value]
            else:
                avg_feat_imp[key].append(value)
                
for key in avg_feat_imp.keys():
    avg_feat_imp[key] = np.sum(avg_feat_imp[key])/len(avg_feat_imp[key])

**Now for the most challenging part, the feature interactions using the contributions of features in each row.**

Idea: form tuples of important features, iterate through all trees to find lists of their split features and see if these features occur near the top in a specific sequence. Maintain a count in dataframe. Features most commonly occuring together should have high interaction.